##   Code for running 3DVAR with the Lorenz 63 model.
#### Code developed by Greg Hakim, Ryan Torn, Aneesh Subramanian.

In [1]:
import numpy as np
import time
from numpy.linalg import inv
import netCDF4 as nc
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import lorenz63_model as lor

###  Data assimilation experiment parameters

In [ ]:
assim_len = 1.0  #  Time between observations
fcst_len  = 2.0  #  Forecast length
nassim    = 200  #  Number of assimilation times
alpha     = 4.e-3  # Alpha control (how fast you try to converge to analysis)

H = np.array([[1., 0., 0.], [0., 0., 0.], [0., 0., 0.]])  #  observation operator
#H = np.array([[1., 0., 0. ]])  #  observation operator for single observation
nobs = len(H[:,0])
R = np.eye(nobs) * 0.25e-2  #  observation error as a diagonal matrix
#R = np.array([[1.0e-2]])  #  observation error for single observation

time1 = time.time()

In [3]:
np.random.seed(0)

bfile = nc.Dataset('L63_B.nc')
invB = inv(bfile.variables['B_matrix'][:,:])
invR = inv(R)

###  Create arrays

In [4]:
xb = np.empty(3)
xf = np.empty(3)
xe = np.empty(3)

yobs  = np.empty(nobs)
innov = np.empty(nobs)

xaerr = np.empty((nassim, 3))
xberr = np.empty((nassim, 3))
xferr = np.empty((nassim, 3))

Jfin = np.empty(nassim)

### IC for truth taken from last time (column vector):

In [5]:
xt = np.array(lor.advance(10., 20., 30., 100.))
## Runs for 100 time units for 100/0.001 steps, starting at x = 10, y =20, z = 30
## Only returns final state

In [6]:
xt

array([ 1.29649403,  2.1041806 , 13.1900553 ])

### Populate initial state by perturbing true state

In [7]:
xa = np.empty(3)
xa = xt[:] + np.random.normal(0, 0.1, 3)
xa

array([ 1.47289926,  2.14419632, 13.2879291 ])

In [8]:
time1 = time.time()
for t in range(nassim):

  #  Advance analysis to next assimilation time
  xb[0], xb[1], xb[2] = lor.advance(xa[0], xa[1], xa[2], assim_len)

  #  Advance the truth, compute observations at the next time
  xt[0], xt[1], xt[2] = lor.advance(xt[0], xt[1], xt[2], assim_len)
  yobs[:] = np.matmul(H,xt) + np.random.normal(0, np.diag(np.sqrt(R)), nobs)

  xa[:] = xb[:]
  niters = 0
  maxiter = 100
  Jold = 1.0e6
  J = 0.
  
  ## Converging towards a state vector with the least cost function
  while abs(Jold - J) > 1.0e-5:

    Jold = J

    #  Compute innovation, background and observation cost function
    innov[:] = yobs[:] - np.matmul(H,xa)
    Jb = 2.0 * np.matmul(np.matmul(np.transpose(xa - xb), invB), xa - xb)
    J0 = 2.0 * np.matmul(np.matmul(np.transpose(innov), invR), innov)
    J = Jb + J0

    print('   cost function = ',J,", Analysis Error:",np.sqrt(np.sum((xa[:]-xt[:])**2)), "Background Error:",np.sqrt(np.sum((xa[:]-xb[:])**2)),"Jb:",Jb, "J0:",J0)

    #  Compute the gradient in the cost function
    gJ = 2.0 * np.matmul(invB,xa - xb) - 2.0 * np.matmul(np.matmul(np.transpose(H),invR),innov)

    #  Compute the new state vector based on cost function gradient
    if niters == 0:
      xa[:] = xa[:] - alpha*gJ[:]
      cgJo = gJ[:]
    else:
      beta = np.matmul(np.transpose(gJ),gJ) / np.matmul(np.transpose(gJo),gJo)
      cgJ = gJ[:] + beta*cgJo[:]
      xa[:] = xa[:] - alpha*cgJ[:]
      cgJo = cgJ[:]

    gJo = gJ[:]

    niters = niters + 1

  print('final cost = ', J, ' after ', niters, ' iterations')

  Jfin[t] = J

  #  Compute analysis and background forecast error
  xberr[t,:] = xb[:] - xt[:]
  xaerr[t,:] = xa[:] - xt[:]

  # compute forecast and error
  xf[0], xf[1], xf[2] = lor.advance(xa[0], xa[1], xa[2], fcst_len)
  xe[0], xe[1], xe[2] = lor.advance(xt[0], xt[1], xt[2], fcst_len)
  xferr[t,:] = xf[:] - xe[:]

print('Analysis Error: ',np.sqrt(sum(sum(xaerr[:,:] * xaerr[:,:])) / float(nassim*3)))
print('Background Error: ',np.sqrt(sum(sum(xberr[:,:] * xberr[:,:])) / float(nassim*3)))
print('Forecast Error: ',np.sqrt(sum(sum(xferr[:,:] * xferr[:,:])) / float(nassim*3)))

time2 = time.time()

print("Total Time:",time2-time1)

   cost function =  294.7655532364859 , Analysis Error: 0.33508046827723703 Background Error: 0.0 Jb: 0.0 J0: 294.7655532364859
   cost function =  21152.772886755152 , Analysis Error: 3.0383699035803744 Background Error: 3.3132743254790986 Jb: 0.21249734861673988 J0: 21152.560389406535
   cost function =  115662949.87824696 , Analysis Error: 219.59267300750884 Background Error: 219.87304022638185 Jb: 935.7981545565796 J0: 115662014.0800924
   cost function =  3357405614951795.0 , Analysis Error: 1182754.3601563664 Background Error: 1182754.640600147 Jb: 27078682361.274227 J0: 3357378536269433.5
   cost function =  2.827883320391541e+30 , Analysis Error: 34326013656506.78 Background Error: 34326013656507.06 Jb: 2.2807881525021004e+25 J0: 2.8278605125100163e+30
   cost function =  2.0062119097073314e+60 , Analysis Error: 2.891219270551838e+28 Background Error: 2.891219270551838e+28 Jb: 1.6180810297489417e+55 J0: 2.0061957288970338e+60
   cost function =  1.0097350692164456e+120 , Analys

/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:24: RuntimeWarning: overflow encountered in matmul
  Jb = 2.0 * np.matmul(np.matmul(np.transpose(xa - xb), invB), xa - xb)
/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:25: RuntimeWarning: overflow encountered in matmul
  J0 = 2.0 * np.matmul(np.matmul(np.transpose(innov), invR), innov)
/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:28: RuntimeWarning: overflow encountered in square
  print('   cost function = ',J,", Analysis Error:",np.sqrt(np.sum((xa[:]-xt[:])**2)), "Background Error:",np.sqrt(np.sum((xa[:]-xb[:])**2)),"Jb:",Jb, "J0:",J0)
/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:38: RuntimeWarning: overflow encountered in matmul
  beta = np.matmul(np.transpose(gJ),gJ) / np.matmul(np.transpose(gJo),gJo)
/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:18: RuntimeWarning: inval

   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost function =  nan , Analysis Error: nan Background Error: nan Jb: nan J0: nan
final cost =  nan  after  1  iterations
   cost 

/var/folders/2d/t7gy2rhs3fn_fgvstmnfq9480000gn/T/ipykernel_76476/1242422260.py:60: RuntimeWarning: overflow encountered in multiply
  print('Analysis Error: ',np.sqrt(sum(sum(xaerr[:,:] * xaerr[:,:])) / float(nassim*3)))
